# CE49X: Introduction to Computational Thinking and Data Science for Civil Engineers
## Week 7: Naive Bayes Classification

**Instructor:** Dr. Eyuphan Koc  
**Department of Civil Engineering, Bogazici University**  
**Semester:** Spring 2026

---

Based on *Python Data Science Handbook* by Jake VanderPlas  
Chapter 5: Machine Learning (Section 5.05 - Naive Bayes Classification)  
[https://jakevdp.github.io/PythonDataScienceHandbook/](https://jakevdp.github.io/PythonDataScienceHandbook/)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
%matplotlib inline

# Set random seed for reproducibility
np.random.seed(42)

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

## Table of Contents

1. [Introduction to Bayesian Classification](#1.-Introduction-to-Bayesian-Classification)
2. [Gaussian Naive Bayes](#2.-Gaussian-Naive-Bayes)
3. [Multinomial Naive Bayes](#3.-Multinomial-Naive-Bayes)
4. [When to Use Naive Bayes](#4.-When-to-Use-Naive-Bayes)
5. [Linear Regression](#5.-Linear-Regression)
6. [Summary and Next Steps](#6.-Summary-and-Next-Steps)

---
## 1. Introduction to Bayesian Classification

### What is Naive Bayes?

**Overview:**
- Naive Bayes classifiers are a family of simple **probabilistic classifiers**
- Based on applying **Bayes' theorem** with strong (naive) independence assumptions
- One of the most straightforward and fast classification algorithms
- Despite naive assumptions, often performs surprisingly well

**Key Applications:**
- Text classification and spam filtering
- Document categorization
- Medical diagnosis
- Sentiment analysis

> **Key Insight: Key Idea**  
> Use probability theory to predict the class label based on feature values

### Bayes's Theorem

**Fundamental Formula:**

$$
P(L \mid \text{features}) = \frac{P(\text{features} \mid L) \cdot P(L)}{P(\text{features})}
$$

**Components:**
- $P(L \mid \text{features})$ = **Posterior probability**: probability of label $L$ given features
- $P(\text{features} \mid L)$ = **Likelihood**: probability of features given label $L$
- $P(L)$ = **Prior probability**: probability of label $L$ before seeing data
- $P(\text{features})$ = **Evidence**: probability of features (normalization constant)

> **Example: For Classification**  
> We choose the label $L$ that maximizes $P(L \mid \text{features})$

### Bayes's Theorem for Classification

**Binary Classification Example:**

For two classes $L_1$ and $L_2$, we compute the posterior ratio:

$$
\frac{P(L_1 \mid \text{features})}{P(L_2 \mid \text{features})} = \frac{P(\text{features} \mid L_1) \, P(L_1)}{P(\text{features} \mid L_2) \, P(L_2)}
$$

**Key Observations:**
- The denominator $P(\text{features})$ cancels out
- We only need to compute the numerator for each class
- Choose the class with the highest posterior probability
- Can extend to multi-class classification naturally

> **Key Insight: Decision Rule**  
> **Predict class** $L_{\text{max}} = \arg\max_L \; P(\text{features} \mid L) \, P(L)$

### Generative vs. Discriminative Models

| | **Generative Models** | **Discriminative Models** |
|---|---|---|
| **Approach** | Model how data is *generated* | Model decision *boundaries* |
| **What they learn** | $P(\text{features} \mid L)$ for each class | $P(L \mid \text{features})$ directly |
| **Can generate samples?** | Yes | No |
| **Examples** | Naive Bayes, GMM, HMM | Logistic regression, SVM, Neural Networks |
| **CE Example** | Learn distributions of structural parameters for each damage level | Learn boundary between safe and unsafe structures directly |

> **Example: Trade-off**  
> Generative models require stronger assumptions but can work with less data

### The "Naive" Assumption

**Feature Independence:**

Naive Bayes assumes that features are **conditionally independent** given the class label:

$$
P(\text{features} \mid L) = P(f_1, f_2, \ldots, f_n \mid L) = P(f_1 \mid L) \cdot P(f_2 \mid L) \cdot \ldots \cdot P(f_n \mid L)
$$

**Why is this "naive"?**
- In reality, features are often correlated
- Example: crack length and deflection in a beam are related
- Ignoring correlations simplifies computation dramatically
- Reduces complexity from exponential to linear in number of features

> **Key Insight: Surprising Result**  
> Despite this strong assumption, Naive Bayes often works well in practice, especially for:
> - Text classification (words are somewhat independent)
> - High-dimensional data (curse of dimensionality helps)

### Complete Naive Bayes Formula

**Putting It All Together:**

For features $\mathbf{x} = (f_1, f_2, \ldots, f_n)$, the posterior probability is:

$$
P(L \mid f_1, \ldots, f_n) \propto P(L) \prod_{i=1}^{n} P(f_i \mid L)
$$

**Classification Procedure:**
1. For each class $L$, compute the prior $P(L)$ from training data
2. For each class $L$ and feature $f_i$, compute $P(f_i \mid L)$
3. For a new sample, multiply prior by all feature likelihoods
4. Choose the class with the highest posterior probability

> **Example: Key Advantage**  
> With $n$ features and $k$ classes, we only need to estimate:
> - $k$ prior probabilities
> - $k \times n$ feature likelihoods (instead of $k \times 2^n$ joint probabilities)

---
## 2. Gaussian Naive Bayes

### Gaussian Naive Bayes: The Idea

**Assumption:**

The likelihood of features given a class follows a **Gaussian (normal) distribution**:

$$
P(f_i \mid L) = \frac{1}{\sqrt{2\pi\sigma_{L}^2}} \exp\left(-\frac{(f_i - \mu_L)^2}{2\sigma_L^2}\right)
$$

**When to use:**
- Features are continuous numerical values
- Features approximately follow normal distributions
- Common in scientific measurements

**Parameters to estimate for each class $L$ and feature $i$:**
- $\mu_{L,i}$ = mean of feature $i$ for class $L$
- $\sigma_{L,i}$ = standard deviation of feature $i$ for class $L$

### Gaussian Naive Bayes: Training

**Training Process:**

Given training data with features $\mathbf{X}$ and labels $\mathbf{y}$:

**Step 1: Compute class priors**

$$
P(L) = \frac{\text{number of samples with label } L}{\text{total number of samples}}
$$

**Step 2: For each class $L$ and feature $i$, compute:**
- Mean: $\mu_{L,i} = \text{mean of feature } i \text{ for samples with label } L$
- Std dev: $\sigma_{L,i} = \text{std dev of feature } i \text{ for samples with label } L$

> **Key Insight: Key Point**  
> Training is just computing sample statistics -- very fast!

### Gaussian Naive Bayes in Scikit-Learn

**Simple Implementation:**

```python
from sklearn.naive_bayes import GaussianNB

# Create the model
model = GaussianNB()

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Get probability estimates
y_prob = model.predict_proba(X_test)
```

**Key Methods:**
- `fit(X, y)`: Train the model on data
- `predict(X)`: Predict class labels for new data
- `predict_proba(X)`: Get probability estimates for each class
- `score(X, y)`: Get accuracy on test data

### [TOGETHER] Example: Structural Health Classification

**Problem Statement:**

Classify structural condition based on inspection measurements:
- Feature 1: Maximum crack length (mm)
- Feature 2: Maximum deflection (mm)
- Feature 3: Vibration frequency (Hz)

Classes: **Good**, **Fair**, **Poor**

**Approach:**
1. Collect labeled inspection data
2. Fit Gaussian distributions for each condition class
3. Use Bayes's theorem to classify new structures
4. Visualize decision boundaries

> **Example: Assumption**  
> Crack length, deflection, and vibration are approximately normally distributed within each condition class (reasonable for structural data)

In [ ]:
# Generate synthetic structural health data
def generate_structural_data(n_samples_per_class=150):
    """
    Generate synthetic structural inspection data.
    
    Returns:
    - X: Feature matrix (crack_length, deflection, vibration)
    - y: Target labels (0=Good, 1=Fair, 2=Poor)
    """
    
    # Good condition: small cracks, small deflection, high frequency
    good_crack = np.random.normal(loc=2.0, scale=0.8, size=n_samples_per_class)
    good_deflection = np.random.normal(loc=5.0, scale=1.5, size=n_samples_per_class)
    good_vibration = np.random.normal(loc=15.0, scale=2.0, size=n_samples_per_class)
    good_data = np.column_stack([good_crack, good_deflection, good_vibration])
    good_labels = np.zeros(n_samples_per_class)
    
    # Fair condition: medium cracks, medium deflection, medium frequency
    fair_crack = np.random.normal(loc=6.0, scale=1.2, size=n_samples_per_class)
    fair_deflection = np.random.normal(loc=12.0, scale=2.0, size=n_samples_per_class)
    fair_vibration = np.random.normal(loc=10.0, scale=1.5, size=n_samples_per_class)
    fair_data = np.column_stack([fair_crack, fair_deflection, fair_vibration])
    fair_labels = np.ones(n_samples_per_class)
    
    # Poor condition: large cracks, large deflection, low frequency
    poor_crack = np.random.normal(loc=12.0, scale=1.5, size=n_samples_per_class)
    poor_deflection = np.random.normal(loc=20.0, scale=2.5, size=n_samples_per_class)
    poor_vibration = np.random.normal(loc=5.0, scale=1.2, size=n_samples_per_class)
    poor_data = np.column_stack([poor_crack, poor_deflection, poor_vibration])
    poor_labels = np.full(n_samples_per_class, 2)
    
    # Combine all data
    X = np.vstack([good_data, fair_data, poor_data])
    y = np.concatenate([good_labels, fair_labels, poor_labels])
    
    # Ensure non-negative values (physical constraint)
    X = np.abs(X)
    
    return X, y

# Generate data
X, y = generate_structural_data(n_samples_per_class=150)

# Create a DataFrame for better visualization
df = pd.DataFrame(X, columns=['Crack Length (mm)', 'Deflection (mm)', 'Vibration (Hz)'])
df['Condition'] = y
df['Condition_Label'] = df['Condition'].map({0: 'Good', 1: 'Fair', 2: 'Poor'})

print("Dataset Shape:", X.shape)
print("\nFirst few rows:")
print(df.head(10))
print("\nClass distribution:")
print(df['Condition_Label'].value_counts())

In [ ]:
# Visualize the data in 2D (crack length vs deflection)
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot 1: Crack Length vs Deflection
for condition, label in [(0, 'Good'), (1, 'Fair'), (2, 'Poor')]:
    mask = df['Condition'] == condition
    axes[0].scatter(df.loc[mask, 'Crack Length (mm)'], 
                   df.loc[mask, 'Deflection (mm)'],
                   label=label, alpha=0.6, s=50)
axes[0].set_xlabel('Crack Length (mm)', fontsize=12)
axes[0].set_ylabel('Deflection (mm)', fontsize=12)
axes[0].set_title('Crack Length vs Deflection', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Crack Length vs Vibration
for condition, label in [(0, 'Good'), (1, 'Fair'), (2, 'Poor')]:
    mask = df['Condition'] == condition
    axes[1].scatter(df.loc[mask, 'Crack Length (mm)'], 
                   df.loc[mask, 'Vibration (Hz)'],
                   label=label, alpha=0.6, s=50)
axes[1].set_xlabel('Crack Length (mm)', fontsize=12)
axes[1].set_ylabel('Vibration (Hz)', fontsize=12)
axes[1].set_title('Crack Length vs Vibration', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Plot 3: Deflection vs Vibration
for condition, label in [(0, 'Good'), (1, 'Fair'), (2, 'Poor')]:
    mask = df['Condition'] == condition
    axes[2].scatter(df.loc[mask, 'Deflection (mm)'], 
                   df.loc[mask, 'Vibration (Hz)'],
                   label=label, alpha=0.6, s=50)
axes[2].set_xlabel('Deflection (mm)', fontsize=12)
axes[2].set_ylabel('Vibration (Hz)', fontsize=12)
axes[2].set_title('Deflection vs Vibration', fontsize=14, fontweight='bold')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nObservations:")
print("- Good condition: small cracks, small deflection, high vibration frequency")
print("- Fair condition: medium values for all features")
print("- Poor condition: large cracks, large deflection, low vibration frequency")
print("- Classes are reasonably well-separated, suitable for Gaussian NB")

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Testing set size: {X_test.shape[0]} samples")
print(f"\nTraining set class distribution:")
unique, counts = np.unique(y_train, return_counts=True)
for cls, cnt in zip(unique, counts):
    print(f"  Class {int(cls)}: {cnt} samples")

In [ ]:
# Create and train the Gaussian Naive Bayes model
gnb_model = GaussianNB()
gnb_model.fit(X_train, y_train)

print("Gaussian Naive Bayes Model trained successfully!")
print("\nModel Parameters:")
print(f"Classes: {gnb_model.classes_}")
print(f"\nClass priors (P(class)):")
for i, prior in enumerate(gnb_model.class_prior_):
    condition_name = ['Good', 'Fair', 'Poor'][i]
    print(f"  {condition_name}: {prior:.3f}")

print(f"\nFeature means for each class (theta):")
feature_names = ['Crack Length', 'Deflection', 'Vibration']
for i, condition in enumerate(['Good', 'Fair', 'Poor']):
    print(f"\n  {condition} condition:")
    for j, feature in enumerate(feature_names):
        print(f"    {feature}: mu={gnb_model.theta_[i, j]:.2f}, sigma^2={gnb_model.var_[i, j]:.2f}")

### Gaussian Naive Bayes: Visualization

The following visualization illustrates how Gaussian NB partitions the feature space. Each class is modeled by its own Gaussian distribution (represented as an ellipse showing the mean and standard deviation), and the decision boundaries separate regions where different classes are most probable.

- **Ellipses** represent Gaussian distributions (mean $\pm$ 2 std dev)
- Each class has its own mean and variance
- Decision boundaries separate regions where different classes are most probable
- Boundaries are **quadratic** (curved), not linear

In [ ]:
# Visualize Gaussian NB: Ellipses for each class and decision boundaries
from matplotlib.patches import Ellipse

fig, ax = plt.subplots(figsize=(10, 7))

# Use first two features: Crack Length and Deflection
X_2d = X[:, :2]
model_2d = GaussianNB()
model_2d.fit(X_2d, y)

# Create mesh for decision boundary
x_min, x_max = X_2d[:, 0].min() - 1, X_2d[:, 0].max() + 1
y_min, y_max = X_2d[:, 1].min() - 1, X_2d[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300),
                     np.linspace(y_min, y_max, 300))
Z = model_2d.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

# Plot decision regions
ax.contourf(xx, yy, Z, alpha=0.2, levels=2, cmap='viridis')
ax.contour(xx, yy, Z, levels=2, colors='gray', linewidths=1, linestyles='dashed')

# Plot data points
colors = ['green', 'orange', 'red']
labels = ['Good', 'Fair', 'Poor']
for i, (color, label) in enumerate(zip(colors, labels)):
    mask = y == i
    ax.scatter(X_2d[mask, 0], X_2d[mask, 1],
              c=color, label=label, edgecolors='black', s=40, alpha=0.6)

# Draw ellipses for each class (mean +/- 2 std)
for i, (color, label) in enumerate(zip(colors, labels)):
    mean = model_2d.theta_[i]
    var = model_2d.var_[i]
    ell = Ellipse(xy=mean, width=4*np.sqrt(var[0]), height=4*np.sqrt(var[1]),
                  edgecolor=color, facecolor='none', linewidth=2, linestyle='--')
    ax.add_patch(ell)

ax.set_xlabel('Crack Length (mm)', fontsize=12)
ax.set_ylabel('Deflection (mm)', fontsize=12)
ax.set_title('Gaussian Naive Bayes: Decision Boundaries and Class Distributions',
             fontsize=14, fontweight='bold')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Decision Boundaries

**Understanding Decision Boundaries:**

The boundary between two classes occurs where their posterior probabilities are equal.

**For Gaussian Naive Bayes:**
- Decision boundaries are **quadratic** (curved) in general
- This is different from linear classifiers (e.g., logistic regression)
- Allows for more flexible decision boundaries
- Can capture some non-linear relationships

> **Example: Mathematical Note**  
> The log-posterior is a quadratic function of features:
>
> $$\log P(L \mid \mathbf{x}) \propto -\sum_i \frac{(f_i - \mu_{L,i})^2}{2\sigma_{L,i}^2} + \log P(L)$$
>
> Setting two classes equal gives a quadratic equation.

### Probability Predictions

**Beyond Binary Classification:**

Naive Bayes provides **probabilistic predictions**, not just class labels.

**Using `predict_proba()`:**
- Returns probability estimates for each class
- Probabilities sum to 1 for each sample
- Useful for ranking predictions by confidence
- Can set custom decision thresholds

**Example output:**

| **Good** | **Fair** | **Poor** |
|:--------:|:--------:|:--------:|
| 0.85     | 0.13     | 0.02     |
| 0.10     | 0.75     | 0.15     |
| 0.02     | 0.28     | 0.70     |

First sample: 85% confident it's "Good".

In [ ]:
# Make predictions on the test set
y_pred = gnb_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Set Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

# Get probability predictions
y_prob = gnb_model.predict_proba(X_test)

print("\nExample predictions (first 10 test samples):")
print(f"{'True':<6} {'Pred':<6} {'P(Good)':<10} {'P(Fair)':<10} {'P(Poor)':<10}")
print("-" * 52)
for i in range(10):
    true_label = ['Good', 'Fair', 'Poor'][int(y_test[i])]
    pred_label = ['Good', 'Fair', 'Poor'][int(y_pred[i])]
    print(f"{true_label:<6} {pred_label:<6} {y_prob[i, 0]:<10.3f} {y_prob[i, 1]:<10.3f} {y_prob[i, 2]:<10.3f}")

In [ ]:
# Compute and visualize confusion matrix
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Good', 'Fair', 'Poor'],
            yticklabels=['Good', 'Fair', 'Poor'],
            cbar_kws={'label': 'Count'})
plt.xlabel('Predicted Condition', fontsize=12, fontweight='bold')
plt.ylabel('True Condition', fontsize=12, fontweight='bold')
plt.title('Confusion Matrix - Gaussian Naive Bayes\nStructural Health Classification', 
          fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Print classification report
print("\nDetailed Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Good', 'Fair', 'Poor']))

In [ ]:
# [PRACTICE] Predict condition for new structural inspections
new_structures = np.array([
    [1.5, 4.0, 16.0],   # Should be Good
    [5.5, 11.0, 10.5],  # Should be Fair
    [13.0, 22.0, 4.5],  # Should be Poor
    [3.0, 8.0, 13.0],   # Borderline Good/Fair
])

predictions = gnb_model.predict(new_structures)
probabilities = gnb_model.predict_proba(new_structures)

print("Predictions for new structures:")
print(f"\n{'Crack':<8} {'Deflect':<8} {'Vibr':<8} {'Predicted':<12} {'P(Good)':<10} {'P(Fair)':<10} {'P(Poor)':<10}")
print("-" * 76)
for i, (structure, pred, prob) in enumerate(zip(new_structures, predictions, probabilities)):
    pred_label = ['Good', 'Fair', 'Poor'][int(pred)]
    print(f"{structure[0]:<8.1f} {structure[1]:<8.1f} {structure[2]:<8.1f} "
          f"{pred_label:<12} {prob[0]:<10.3f} {prob[1]:<10.3f} {prob[2]:<10.3f}")

### Gaussian Naive Bayes: Strengths and Limitations

| **Strengths** | **Limitations** |
|---|---|
| Very fast training and prediction | Assumes features are independent |
| Works well with small datasets | Assumes Gaussian distributions |
| Provides probabilistic predictions | Can be outperformed by more complex models |
| No hyperparameters to tune | Sensitive to feature scaling |
| Handles multi-class naturally | Poor if assumptions violated |
| Interpretable (can examine means/stds) | |

> **Key Insight: Best Practices**  
> - Check if features are approximately Gaussian (histograms, Q-Q plots)
> - Consider transforming features (log, Box-Cox) if heavily skewed
> - Compare with other algorithms on your specific dataset

---
## 3. Multinomial Naive Bayes

### Multinomial Naive Bayes: The Idea

**Assumption:**

Features represent **counts** or **frequencies** of events:

$$
P(f_i \mid L) = \frac{\text{count of feature } i \text{ in class } L}{\text{total count of all features in class } L}
$$

**When to use:**
- Features are discrete counts (e.g., word frequencies)
- Text classification (spam detection, document categorization)
- Any count-based data (e.g., website click counts)

**Key difference from Gaussian NB:**
- Uses multinomial distribution instead of Gaussian
- Features must be non-negative (counts)
- Common in natural language processing

### Text Classification with Multinomial NB

**The Challenge:**

How do we represent text documents as numerical features?

**Common approach: Bag of Words**
1. Create a vocabulary of all unique words
2. For each document, count occurrences of each word
3. Represent document as a vector of word counts
4. Ignore word order (hence "bag" of words)

**Example:**
- Document 1: "the bridge is safe"
- Document 2: "the building is safe"
- Vocabulary: [the, bridge, building, is, safe]
- Doc 1 vector: [1, 1, 0, 1, 1]
- Doc 2 vector: [1, 0, 1, 1, 1]

### TF-IDF: Improving Bag of Words

**Problem with Raw Counts:**

Common words ("the", "is", "a") dominate but carry little meaning.

**TF-IDF (Term Frequency - Inverse Document Frequency):**

Weight words by both frequency and rarity:

$$
\text{TF-IDF}(t, d) = \text{TF}(t, d) \times \text{IDF}(t)
$$

where:
- $\text{TF}(t, d)$ = frequency of term $t$ in document $d$
- $\text{IDF}(t) = \log\frac{\text{total documents}}{\text{documents containing } t}$

**Effect:**
- Common words (high document frequency) get low weight
- Rare but meaningful words get high weight
- Better feature representation for classification

### Multinomial NB in Scikit-Learn

**Implementation with TF-IDF:**

```python
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

# Create a pipeline: vectorization -> classification
model = make_pipeline(
    TfidfVectorizer(),
    MultinomialNB()
)

# Train the model
model.fit(train_data, train_labels)

# Make predictions
predictions = model.predict(test_data)
```

**Pipeline advantages:** Combines preprocessing and classification into one object.

### [TOGETHER] Example: 20 Newsgroups Dataset

**Dataset Description:**

Classic text classification dataset with 20 different newsgroup topics.

**Sample Categories:**
- `talk.religion.misc` -- Religious discussions
- `soc.religion.christian` -- Christianity discussions
- `sci.space` -- Space science and exploration
- `comp.graphics` -- Computer graphics

**Classification Task:**
- Given a text document, predict which newsgroup it belongs to
- ~18,000 documents total
- Highly diverse vocabulary
- Good benchmark for text classification algorithms

In [ ]:
# Select 4 categories for classification
categories = ['talk.religion.misc',
              'soc.religion.christian',
              'sci.space',
              'comp.graphics']

# Load training data
train_data = fetch_20newsgroups(subset='train', 
                                categories=categories,
                                remove=('headers', 'footers', 'quotes'),
                                random_state=42)

# Load test data
test_data = fetch_20newsgroups(subset='test', 
                               categories=categories,
                               remove=('headers', 'footers', 'quotes'),
                               random_state=42)

print(f"Training samples: {len(train_data.data)}")
print(f"Testing samples: {len(test_data.data)}")
print(f"\nCategories: {train_data.target_names}")
print(f"\nTraining set class distribution:")
unique, counts = np.unique(train_data.target, return_counts=True)
for i, (cls, cnt) in enumerate(zip(unique, counts)):
    print(f"  {train_data.target_names[i]}: {cnt} samples")

In [ ]:
# Display sample documents from each category
print("Sample documents from each category:\n")
for i, category in enumerate(train_data.target_names):
    # Find first document of this category
    idx = np.where(train_data.target == i)[0][0]
    print(f"Category: {category}")
    print(f"Document preview (first 300 chars):")
    print(train_data.data[idx][:300])
    print("\n" + "="*80 + "\n")

In [ ]:
# Create a pipeline with TF-IDF vectorization and Multinomial NB
text_model = make_pipeline(
    TfidfVectorizer(max_features=5000, stop_words='english'),
    MultinomialNB(alpha=0.1)
)

# Train the model
print("Training the text classifier...")
text_model.fit(train_data.data, train_data.target)
print("Training complete!")

# Make predictions on test set
y_pred_text = text_model.predict(test_data.data)

# Calculate accuracy
accuracy_text = accuracy_score(test_data.target, y_pred_text)
print(f"\nTest Set Accuracy: {accuracy_text:.4f} ({accuracy_text*100:.2f}%)")

### Confusion Matrix

**Evaluating Multi-Class Classification:**

A confusion matrix shows true vs. predicted labels for all classes.

**Reading the matrix:**
- Diagonal = correct predictions (darker in heatmap)
- Off-diagonal = misclassifications
- Each row sums to total samples for that true class

In [ ]:
# Compute and visualize confusion matrix
cm_text = confusion_matrix(test_data.target, y_pred_text)

plt.figure(figsize=(10, 8))
sns.heatmap(cm_text, annot=True, fmt='d', cmap='YlOrRd',
            xticklabels=['religion.misc', 'christian', 'space', 'graphics'],
            yticklabels=['religion.misc', 'christian', 'space', 'graphics'],
            cbar_kws={'label': 'Count'})
plt.xlabel('Predicted Category', fontsize=12, fontweight='bold')
plt.ylabel('True Category', fontsize=12, fontweight='bold')
plt.title('Confusion Matrix - Multinomial Naive Bayes\n20 Newsgroups Classification', 
          fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Print classification report
print("\nDetailed Classification Report:")
print(classification_report(test_data.target, y_pred_text, 
                          target_names=train_data.target_names))

### [LIVE] Making Predictions on New Text

In [ ]:
# Function to predict category for custom text
def predict_category(text, model, target_names):
    """
    Predict the category of a given text.
    """
    pred = model.predict([text])[0]
    prob = model.predict_proba([text])[0]
    
    print(f"Text: '{text[:100]}...'" if len(text) > 100 else f"Text: '{text}'")
    print(f"Predicted category: {target_names[pred]}")
    print(f"Probabilities:")
    for name, p in zip(target_names, prob):
        print(f"  {name}: {p:.4f}")
    print()

# Test with custom strings
custom_texts = [
    "NASA launched a new space telescope to explore distant galaxies and exoplanets",
    "The Bible teaches us about love, forgiveness, and the path to salvation",
    "OpenGL and DirectX are popular graphics APIs for rendering 3D scenes",
    "What is the meaning of life according to different philosophical traditions?",
    "The Mars rover discovered evidence of ancient water on the red planet",
    "Ray tracing algorithms produce photorealistic images by simulating light physics"
]

print("Predictions for custom text samples:\n")
print("=" * 80 + "\n")
for text in custom_texts:
    predict_category(text, text_model, train_data.target_names)
    print("=" * 80 + "\n")

In [ ]:
# Analyze most informative features
vectorizer = text_model.named_steps['tfidfvectorizer']
classifier = text_model.named_steps['multinomialnb']

feature_names_text = np.array(vectorizer.get_feature_names_out())

# For each category, show top 15 most informative words
print("Top 15 most informative words for each category:\n")
for i, category in enumerate(train_data.target_names):
    # Get log probabilities for this category
    log_prob = classifier.feature_log_prob_[i]
    
    # Get top 15 features
    top_indices = np.argsort(log_prob)[-15:][::-1]
    top_features = feature_names_text[top_indices]
    
    print(f"{category}:")
    print(f"  {', '.join(top_features)}")
    print()

### Multinomial NB: Strengths and Limitations

| **Strengths** | **Limitations** |
|---|---|
| Excellent for text classification | Assumes feature independence (violated in text) |
| Very fast training and prediction | Ignores word order and context |
| Works well with high-dimensional sparse data | Requires non-negative features |
| Requires relatively little training data | Can be beaten by deep learning for very large datasets |
| Handles multi-class naturally | Sensitive to feature representation |
| Provides probability estimates | |

> **Key Insight: Practical Tips**  
> - Use TF-IDF instead of raw counts for better performance
> - Remove very common and very rare words (stop words, rare terms)
> - Consider n-grams (word pairs, triples) to capture some context

### Laplace Smoothing

**The Zero Probability Problem:**

What if a word never appears in training data for a particular class?

$$
P(\text{"bridge"} \mid \text{space}) = 0
$$

This would make the entire posterior probability zero!

**Solution: Laplace (Additive) Smoothing:**

Add a small count (usually 1) to all feature counts:

$$
P(f_i \mid L) = \frac{\text{count}(f_i, L) + \alpha}{\text{total count}(L) + \alpha \cdot n\_features}
$$

where $\alpha$ is the smoothing parameter (default: $\alpha = 1$).

**Effect:**
- No feature has zero probability
- Prevents numerical instability
- Controlled by `alpha` parameter in scikit-learn

### Model Comparison: Gaussian NB vs. Multinomial NB

In [ ]:
# Create a comparison summary
comparison_data = {
    'Model': ['Gaussian NB (Structural)', 'Multinomial NB (Text)'],
    'Dataset': ['Structural Health (450 samples)', '20 Newsgroups (~2400 samples)'],
    'Features': ['3 continuous features', '5000 TF-IDF features'],
    'Classes': ['3 (Good/Fair/Poor)', '4 (newsgroup categories)'],
    'Accuracy': [f'{accuracy:.4f}', f'{accuracy_text:.4f}']
}

comparison_df = pd.DataFrame(comparison_data)
print("Model Comparison Summary:\n")
print(comparison_df.to_string(index=False))

print("\n" + "="*80)
print("\nKey Observations:")
print("- Gaussian NB works well for continuous numerical features")
print("- Multinomial NB excels at text classification with count-based features")
print("- Both models are fast, interpretable, and provide probabilistic predictions")
print("- Choice of variant depends on the nature of your features")

---
## 4. When to Use Naive Bayes

### Advantages of Naive Bayes

**Why Choose Naive Bayes?**

**1. Speed**
- Training: Just compute statistics (means, counts)
- Prediction: Simple multiplication and comparison
- Scales well to very large datasets

**2. Simplicity**
- Few or no hyperparameters to tune
- Easy to implement and understand
- Straightforward interpretation

**3. Probabilistic Predictions**
- Provides probability estimates, not just labels
- Useful for ranking or setting decision thresholds
- Natural uncertainty quantification

**4. Works with Limited Data**
- Can train with relatively small datasets
- Strong assumptions reduce data requirements

### When Naive Bayes Works Well

**Ideal Scenarios:**

**1. High-Dimensional Data**
- Text classification (thousands of features)
- As dimensionality increases, classes become more separated
- Independence assumption matters less

**2. Well-Separated Classes**
- When class distributions don't overlap much
- Decision boundaries are not critical

**3. Need for Baseline**
- Quick baseline before trying complex models
- Fast prototyping and iteration

**4. Real-Time Predictions**
- When speed is critical
- Spam filtering, sentiment analysis

### Limitations and When NOT to Use

**Cases Where Naive Bayes Struggles:**

**1. Strong Feature Dependencies**
- Violates independence assumption
- Example: height and weight are correlated
- Better: use models that capture correlations

**2. Complex Decision Boundaries**
- Gaussian NB has quadratic boundaries
- Cannot learn arbitrary non-linear boundaries
- Better: SVM with RBF kernel, neural networks

**3. Continuous Features with Non-Gaussian Distribution**
- Heavy-tailed, multi-modal, or skewed distributions
- Better: transform features or use different model

**4. When You Need the Best Possible Accuracy**
- Ensemble methods (Random Forest, Gradient Boosting) usually better
- Deep learning for very large datasets

### Naive Bayes vs. Other Classifiers

| **Aspect** | **Naive Bayes** | **Logistic Regression** | **SVM** |
|---|---|---|---|
| Speed | Very fast | Fast | Moderate |
| Accuracy | Good | Good to very good | Very good |
| High-dim data | Excellent | Good | Good |
| Interpretability | High | High | Medium |
| Probability | Native | Native | Calibration needed |
| Assumptions | Strong | Moderate | Weak |
| Hyperparameters | Very few | Few | Several |

> **Key Insight: Recommendation**  
> Start with Naive Bayes as a baseline, then try more complex models if needed.

### The Curse of Dimensionality (Helps NB!)

**Surprising Advantage:**

In high dimensions, Naive Bayes often performs better than expected.

**Why?**
- As dimensions increase, data points become more separated
- Classes occupy distinct regions of feature space
- Simple decision boundaries work better
- Feature correlations matter less

**Example: Text Classification**
- Vocabulary size: 10,000+ words
- Very high-dimensional sparse vectors
- Documents from different topics use different words
- Even with naive independence, classification works well

> **Example: Key Insight**  
> Sometimes simple assumptions + high dimensions > complex models + low dimensions

### Practical Guidelines

**Decision Framework for Using Naive Bayes:**

**Use Naive Bayes when:**
- You need a quick baseline or prototype
- You have text or count data (Multinomial NB)
- You have continuous features that are roughly Gaussian (Gaussian NB)
- Speed is critical (real-time predictions)
- You have limited training data
- You need probability estimates
- Interpretability is important

**Consider alternatives when:**
- Features are highly correlated
- You need maximum accuracy (competitions, critical applications)
- Decision boundaries are very complex
- You have lots of data and computational resources

### Variants of Naive Bayes in Scikit-Learn

**Different Types for Different Data:**

**1. GaussianNB**
- Continuous features following Gaussian distribution
- Example: sensor measurements, physical properties

**2. MultinomialNB**
- Discrete counts or frequencies
- Example: word counts in text, event counts

**3. BernoulliNB**
- Binary features (0/1, True/False)
- Example: word presence/absence in document

**4. ComplementNB**
- Modified version of MultinomialNB
- Better for imbalanced datasets

---
## 5. Linear Regression

### From Classification to Regression

**Bridging Two Fundamental Tasks:**

We have just studied Naive Bayes, a powerful **classification** algorithm. Now we transition to **Linear Regression**, the most fundamental **regression** algorithm. Both are essential baseline models in a civil engineer's toolkit.

| | **Naive Bayes (Classification)** | **Linear Regression (Regression)** |
|---|---|---|
| **Task** | Predict discrete class labels | Predict continuous numerical values |
| **Output** | Categories (e.g., Good / Fair / Poor) | Real numbers (e.g., 12.7 mm) |
| **Foundation** | Bayes' theorem | Least squares optimization |
| **Model type** | Generative | Discriminative |
| **Key strength** | Fast probabilistic classification | Interpretable coefficient estimates |

> **Key Insight: Complementary Tools**  
> Both are simple, fast, interpretable baseline models for their respective tasks. Naive Bayes classifies structural condition (Good/Fair/Poor), while Linear Regression predicts exact deflection in mm.

### What is Linear Regression?

**Overview (extending concepts from Week 06):**

Linear Regression is arguably the **oldest and most widely used** statistical technique. We introduced the basics in Week 06; here we extend those concepts to more powerful variants.

**Key Characteristics:**
- One of the oldest statistical methods (Legendre, 1805; Gauss, 1809)
- Extremely fast to train -- has a closed-form solution
- Highly interpretable: each coefficient tells you the effect of one feature
- Serves as the **baseline** against which more complex regression models are compared

**Civil Engineering Applications:**
- Predicting structural responses (deflection, stress) from loads
- Estimating material properties (strength vs. water-cement ratio)
- Modeling traffic flow (volume vs. time of day, lane count)
- Cost estimation (project cost vs. area, number of floors)

> **Example: Why Linear Regression Matters**  
> In structural design, engineers routinely use linear models to predict beam deflection from applied load, span length, and cross-section properties. The model coefficients directly correspond to physical parameters.

### Simple Linear Regression Review

**The Simplest Case: One Feature**

$$y = ax + b$$

where $a$ is the slope and $b$ is the intercept.

**Ordinary Least Squares (OLS):**

The OLS approach finds the line that minimizes the sum of squared residuals:

$$\text{minimize: } \sum_{i=1}^{n} (y_i - (ax_i + b))^2$$

**Key Properties:**
- Has a **unique closed-form solution** -- no iterative optimization needed
- The solution minimizes the total squared distance from data points to the line
- Computationally efficient: solved via matrix operations in $O(np^2)$ time
- Residuals (errors) sum to zero when an intercept is included

> **Example: Civil Engineering Interpretation**  
> If we model beam deflection as $\delta = a \cdot P + b$, then $a$ tells us how many mm the beam deflects per additional kN of load, and $b$ is the initial deflection under self-weight.

In [ ]:
# Simple Linear Regression -- quick review
from sklearn.linear_model import LinearRegression

# Generate sample data: y = 2*x - 5 + noise
rng = np.random.RandomState(1)
x = 10 * rng.rand(50)
y = 2 * x - 5 + rng.randn(50)

# Create and train the model
model = LinearRegression(fit_intercept=True)
model.fit(x[:, np.newaxis], y)

# Predictions for plotting
xfit = np.linspace(0, 10, 1000)
yfit = model.predict(xfit[:, np.newaxis])

print(f"Slope (a):     {model.coef_[0]:.4f}")    # ~2.0
print(f"Intercept (b): {model.intercept_:.4f}")   # ~-5.0

# Plot
plt.figure(figsize=(10, 6))
plt.scatter(x, y, c='indianred', s=50, edgecolors='k', alpha=0.7, label='Data')
plt.plot(xfit, yfit, 'steelblue', linewidth=2, label='Linear Fit')
plt.xlabel('x', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title('Simple Linear Regression', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Multiple Linear Regression

**Extending to Multiple Features:**

$$y = a_0 + a_1 x_1 + a_2 x_2 + \cdots + a_n x_n$$

**Geometric Interpretation:**
- With 1 feature: fitting a **line** in 2D
- With 2 features: fitting a **plane** in 3D
- With $n$ features: fitting a **hyperplane** in $(n+1)$-dimensional space

> **Example: CE Application**  
> Predicting beam deflection from multiple factors:
>
> $$\delta = a_0 + a_1 \cdot \text{load} + a_2 \cdot \text{span} + a_3 \cdot \text{moment\_of\_inertia}$$
>
> Each coefficient $a_i$ quantifies the independent effect of that variable on deflection.

**Important Note:**

"Linear" in Linear Regression refers to **linearity in the coefficients** $a_i$, not in the features $x_i$. This distinction becomes crucial when we discuss basis function regression below.

In [ ]:
# Multiple Linear Regression with 3 features
rng = np.random.RandomState(1)
X_multi = 10 * rng.rand(100, 3)  # 100 samples, 3 features
y_multi = 0.5 + np.dot(X_multi, [1.5, -2., 1.])  # true coefficients

model_multi = LinearRegression()
model_multi.fit(X_multi, y_multi)

print(f"Intercept: {model_multi.intercept_:.4f}")   # ~0.5
print(f"Coefficients: {model_multi.coef_}")           # ~[1.5, -2., 1.]
print("\nLinear regression perfectly recovers the true coefficients!")

### Basis Function Regression

**Adapting Linear Regression to Nonlinear Relationships:**

The key idea is to **transform the input features** using basis functions $f_n(x)$, then fit a linear model in the transformed feature space.

**General Form:**

$$y = a_0 + a_1 f_1(x) + a_2 f_2(x) + a_3 f_3(x) + \cdots$$

**Why this works:**
- The model is still **linear in the coefficients** $a_i$
- But can capture **nonlinear relationships** between $x$ and $y$
- All the fast linear algebra machinery of OLS still applies
- The choice of basis functions encodes our assumptions about the data

**Common Basis Functions:**
- **Polynomial:** $f_n(x) = x^n$ (global influence)
- **Gaussian:** $f_n(x) = \exp(-(x - \mu_n)^2 / 2\sigma^2)$ (local influence)
- **Fourier:** $f_n(x) = \sin(n \omega x)$ or $\cos(n \omega x)$ (periodic patterns)

> **Key Insight: Power of Basis Functions**  
> By choosing appropriate basis functions, we can fit complex nonlinear patterns while still using fast linear regression algorithms.

### Polynomial Basis Functions

**If $f_n(x) = x^n$, the model becomes a polynomial:**

$$y = a_0 + a_1 x + a_2 x^2 + a_3 x^3 + \cdots + a_d x^d$$

**Degree $d$ controls model flexibility:**
- $d = 1$: straight line (underfitting if data is curved)
- $d = 2$: parabola
- $d = 7$: highly flexible curve
- $d = n-1$: passes through every data point (overfitting)

**Feature Transformation Example (degree 3):**

| **Input** | **Transformed Features** |
|---|---|
| $x = 2$ | $[1, 2, 4, 8]$ |
| $x = 3$ | $[1, 3, 9, 27]$ |

Built into scikit-learn as `PolynomialFeatures`. Note that a degree $d$ polynomial on $n$ features creates $\binom{n+d}{d}$ new features, which can grow rapidly.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# Generate nonlinear data (sine wave with noise)
rng = np.random.RandomState(1)
x = 10 * rng.rand(50)
y = np.sin(x) + 0.1 * rng.randn(50)

xfit = np.linspace(0, 10, 1000)

# Create a 7th-degree polynomial model using pipeline
poly_model = make_pipeline(
    PolynomialFeatures(degree=7),
    LinearRegression()
)

poly_model.fit(x[:, np.newaxis], y)
yfit = poly_model.predict(xfit[:, np.newaxis])

plt.figure(figsize=(10, 6))
plt.scatter(x, y, c='indianred', s=50, edgecolors='k', alpha=0.7, label='Data')
plt.plot(xfit, yfit, 'steelblue', linewidth=2, label='Degree 7 Polynomial')
plt.xlabel('x', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title('Polynomial Basis Function Regression', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Pipeline advantage: Combines preprocessing and modeling in one object")

### Gaussian Basis Functions

**An Alternative to Polynomials:**

Each basis function is a bell-shaped Gaussian centered at $\mu_i$:

$$f_i(x) = \exp\left(-\frac{(x - \mu_i)^2}{2\sigma^2}\right)$$

**Model Form:**

$$y = \sum_{i} a_i \cdot \exp\left(-\frac{(x - \mu_i)^2}{2\sigma^2}\right)$$

**Properties:**
- **Local influence:** each basis only affects predictions near its center $\mu_i$
- **Width controlled by $\sigma$:** larger $\sigma$ means broader, smoother bases
- **Smoother than high-degree polynomials:** no wild oscillations at boundaries
- **Uniform spacing:** centers $\mu_i$ are typically spread evenly across the data range

> **Key Insight: Local vs. Global**  
> Gaussian bases provide local, smooth approximations without the wild oscillations of high-degree polynomials. This makes them particularly useful for modeling structural response curves that vary smoothly across their domain.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class GaussianFeatures(BaseEstimator, TransformerMixin):
    """Uniformly spaced Gaussian features for 1D input."""
    
    def __init__(self, N, width_factor=2.0):
        self.N = N
        self.width_factor = width_factor
    
    def fit(self, X, y=None):
        # Create N centers along data range
        self.centers_ = np.linspace(X.min(), X.max(), self.N)
        self.width_ = self.width_factor * (self.centers_[1] - self.centers_[0])
        return self
    
    def transform(self, X):
        # Apply Gaussian basis functions
        return np.exp(-0.5 * ((X - self.centers_) / self.width_) ** 2)

# Demonstrate with 20 Gaussian bases
gauss_model = make_pipeline(GaussianFeatures(20), LinearRegression())
gauss_model.fit(x[:, np.newaxis], y)
yfit_gauss = gauss_model.predict(xfit[:, np.newaxis])

plt.figure(figsize=(10, 6))
plt.scatter(x, y, c='indianred', s=50, edgecolors='k', alpha=0.7, label='Data')
plt.plot(xfit, yfit_gauss, 'steelblue', linewidth=2, label='20 Gaussian Bases')
plt.xlabel('x', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title('Gaussian Basis Function Regression', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### The Problem of Overfitting

**When the Model is Too Flexible:**

With too many basis functions, the model becomes overly flexible and starts fitting the **noise** in the data rather than the underlying pattern.

**Symptoms of Overfitting:**
- Excellent training performance but poor test performance
- Wild oscillations between data points
- Very large coefficient values (positive and negative)
- Model predictions are unstable -- small changes in data cause large changes in predictions

> **Example: CE Context**  
> Imagine fitting a 20th-degree polynomial to 25 soil bearing capacity measurements. The curve would pass near every point but oscillate wildly between them, giving nonsensical predictions at new locations.

**The Solution: Regularization**

Add a **penalty term** to the loss function that discourages large coefficients. This constrains the model's flexibility and improves generalization to unseen data.

In [ ]:
# Demonstrate overfitting with too many Gaussian bases
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

n_bases_list = [10, 20, 30]
titles = ['10 Bases (Good)', '20 Bases (Good)', '30 Bases (Overfitting)']

for ax, n_bases, title in zip(axes, n_bases_list, titles):
    gauss = make_pipeline(GaussianFeatures(n_bases), LinearRegression())
    gauss.fit(x[:, np.newaxis], y)
    yfit_g = gauss.predict(xfit[:, np.newaxis])
    
    ax.scatter(x, y, c='indianred', s=40, edgecolors='k', alpha=0.7)
    ax.plot(xfit, yfit_g, 'steelblue', linewidth=2)
    ax.set_xlabel('x', fontsize=11)
    ax.set_ylabel('y', fontsize=11)
    ax.set_title(title, fontsize=13, fontweight='bold')
    ax.set_ylim(-1.5, 1.5)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Ridge Regression (L2 Regularization)

**Penalizing Large Coefficients:**

Ridge regression adds a penalty proportional to the **sum of squared coefficients**:

$$\text{Loss} = \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 + \alpha \sum_{j=1}^{p} \theta_j^2$$

**The Regularization Parameter $\alpha$:**
- $\alpha = 0$: standard linear regression (no regularization)
- Small $\alpha$: mild regularization, coefficients slightly shrunk
- Large $\alpha$: strong regularization, coefficients shrink toward zero
- $\alpha \to \infty$: all coefficients approach zero (underfitting)

**Key Properties:**
- Can be computed as efficiently as standard linear regression (closed-form solution)
- All coefficients are shrunk but **none become exactly zero**
- Also known as **Tikhonov regularization** in applied mathematics
- Equivalent to adding a Gaussian prior on the coefficients (Bayesian interpretation)

In [ ]:
from sklearn.linear_model import Ridge

# Ridge regression tames overfitting
ridge_model = make_pipeline(GaussianFeatures(30), Ridge(alpha=0.1))
ridge_model.fit(x[:, np.newaxis], y)
yfit_ridge = ridge_model.predict(xfit[:, np.newaxis])

plt.figure(figsize=(10, 6))
plt.scatter(x, y, c='indianred', s=50, edgecolors='k', alpha=0.7, label='Data')
plt.plot(xfit, yfit_ridge, 'steelblue', linewidth=2, label='Ridge (α=0.1, 30 bases)')
plt.xlabel('x', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title('Ridge Regression: Taming Overfitting', fontsize=14)
plt.legend(fontsize=11)
plt.ylim(-1.5, 1.5)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Ridge regression produces a smooth fit even with 30 basis functions!")

### Lasso Regression (L1 Regularization)

**An Alternative Penalty Using Absolute Values:**

$$\text{Loss} = \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 + \alpha \sum_{j=1}^{p} |\theta_j|$$

**Key Difference from Ridge:**

Lasso drives some coefficients to **exactly zero**, effectively performing **automatic feature selection**. The resulting model is **sparse** -- it uses only a subset of the available features.

**Why Does L1 Produce Zeros?**
- The absolute value function has a "corner" at zero
- During optimization, coefficients are pushed toward zero and can reach it exactly
- Ridge (L2) only shrinks coefficients -- they approach zero but never reach it

> **Example: CE Application**  
> When predicting bridge condition from 50 sensor measurements, Lasso automatically identifies the 5-10 most informative sensors, simplifying both the model and the monitoring system.

In [ ]:
from sklearn.linear_model import Lasso

# Lasso regression: automatic feature selection
lasso_model = make_pipeline(GaussianFeatures(30), Lasso(alpha=0.001))
lasso_model.fit(x[:, np.newaxis], y)
yfit_lasso = lasso_model.predict(xfit[:, np.newaxis])

# Check sparsity
coefficients = lasso_model.named_steps['lasso'].coef_
n_nonzero = np.sum(coefficients != 0)

plt.figure(figsize=(10, 6))
plt.scatter(x, y, c='indianred', s=50, edgecolors='k', alpha=0.7, label='Data')
plt.plot(xfit, yfit_lasso, 'steelblue', linewidth=2, 
         label=f'Lasso (α=0.001, {n_nonzero}/{len(coefficients)} bases used)')
plt.xlabel('x', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title('Lasso Regression: Automatic Feature Selection', fontsize=14)
plt.legend(fontsize=11)
plt.ylim(-1.5, 1.5)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Non-zero coefficients: {n_nonzero} / {len(coefficients)}")
print("Lasso automatically selected the most relevant basis functions!")

### Ridge vs. Lasso: Choosing the Right Regularization

| | **Ridge (L2)** | **Lasso (L1)** |
|---|---|---|
| **Penalty** | $\alpha \sum \theta_j^2$ | $\alpha \sum |\theta_j|$ |
| **Coefficients** | All non-zero (shrunk) | Many exactly zero (sparse) |
| **Feature selection** | No -- keeps all features | Yes -- automatic selection |
| **Best when** | Many features all contribute | Few features truly matter |
| **Correlated features** | Shares weight among them | Picks one, zeros out others |
| **Solution** | Closed-form | Iterative (coordinate descent) |
| **Scikit-learn** | `Ridge(alpha=...)` | `Lasso(alpha=...)` |

**Elastic Net: The Best of Both Worlds**

Combines L1 and L2 penalties:

$$\text{Loss} = \sum (y_i - \hat{y}_i)^2 + \alpha_1 \sum |\theta_j| + \alpha_2 \sum \theta_j^2$$

Available as `ElasticNet` in scikit-learn. Useful when you want some sparsity but also want to handle correlated features gracefully.

> **Key Insight: Practical Guidance**  
> Start with Ridge if you have no reason to expect sparsity. Use Lasso if you suspect only a few features matter. Use Elastic Net when features are correlated and you want some sparsity.

---
## 6. Summary and Next Steps

### Summary: Naive Bayes Classification

**What We Learned:**

1. **Naive Bayes Fundamentals**
   - Based on Bayes's theorem with naive independence assumption
   - Generative model that learns $P(\text{features} \mid \text{class})$
   - Fast, simple, and effective for many problems

2. **Gaussian Naive Bayes**
   - Assumes features follow Gaussian (normal) distributions
   - Suitable for continuous numerical features
   - Applied to structural health classification

3. **Multinomial Naive Bayes**
   - Designed for discrete count data
   - Excellent for text classification
   - Used with TF-IDF feature extraction

4. **Key Strengths**
   - Very fast training and prediction
   - Works well with high-dimensional data
   - Provides probabilistic predictions
   - Few hyperparameters to tune

5. **Limitations**
   - Assumes feature independence (often violated)
   - Can be outperformed by more complex models
   - Sensitive to feature distribution assumptions

### Next Steps

**Building on These Foundations:**

**Topics to Explore:**
- Logistic Regression (discriminative alternative to Naive Bayes)
- Support Vector Machines (more flexible decision boundaries)
- Decision Trees and Ensemble Methods (Random Forest, Gradient Boosting)
- Deep Learning for text (Word embeddings, Transformers)
- Advanced regression: Generalized Linear Models (GLMs)
- Time series modeling and forecasting

**Practice Exercises:**
- Implement Gaussian NB on civil engineering datasets
- Build polynomial regression for nonlinear structural behavior
- Compare regularization methods (Ridge, Lasso, Elastic Net)
- Feature engineering for real-world prediction problems
- Cross-validation and hyperparameter tuning

---

**References:**
- Python Data Science Handbook by Jake VanderPlas, Chapter 5.05
- Scikit-learn documentation: https://scikit-learn.org/stable/modules/naive_bayes.html
- 20 Newsgroups dataset: http://qwone.com/~jason/20Newsgroups/

---

**End of Week 07 Lecture Notebook**